In [1]:
from IPython.core.display import display, HTML
display(HTML('<style>.container { width:100% !important; }</style>'))

In [161]:
import numpy as np
import matplotlib.pyplot as plt
import tqdm.notebook as tqdm

In [310]:
with open('input.txt') as f:
    N = int(f.readline())
    task_complexity = np.array(list(map(int, f.readline().split())))
    execution_time = np.array(list(map(float, f.readline().split())))
    M, dev_coefs = int(f.readline()), np.array([list(map(float, line.split())) for line in f.readlines()])

In [336]:
class TaskDist:
    
    rng = np.random.default_rng()
    
    def __init__(self, complexity: np.ndarray, exec_time: np.ndarray, dev_coefs: np.ndarray, dev_count: int, population_size: int, individ_size: int, k_best: int, tourn_count: int, tourn_size: int, tourn_p: float,
                 mutation_prob: float):
        self.complexity = complexity
        self.exec_time = exec_time
        self.dev_coefs = dev_coefs
        self.dev_count = dev_count
        self.population_size = population_size
        self.individ_size = individ_size
        self.k_best = k_best
        self.tourn_count = tourn_count
        self.tourn_size = tourn_size
        self.tourn_p = tourn_p
        self.mutation_prob = mutation_prob
        self._fitness = None
        self.population = np.array([TaskDist.rng.integers(dev_count, size=individ_size) for _ in range(population_size)])
        
    def fitness(self) -> np.ndarray:
        if self._fitness is not None:
            return self._fitness
        
        self._fitness = np.array([np.max([self.dev_coefs[j][self.complexity[self.population[i] == j] - 1] @ self.exec_time[self.population[i] == j] for j in range(self.dev_count)]) for i in range(self.population_size)])
        
#         for i in range(self.population):
#             dev_time = np.max([self.dev_coefs[j][self.complexity[self.population[i] == j] - 1] @ self.exec_time[self.population[i] == j] for j in range(self.dev_count)])
        
#         self._fitness = np.array([self.dev_coefs[self.population[i]][np.arange(N), (self.complexity - 1)] @ self.exec_time for i in range(self.population.shape[0])])
        
        return self._fitness
    
    def selection(self):
        new_inds = np.zeros(self.k_best + self.tourn_count, dtype=int)
        free_inds = np.ones(self.population_size, dtype=int)
        
        if self.k_best > 0:
            inds = np.argsort(self.fitness())[:self.k_best]
            new_inds[-self.k_best:] = inds
            free_inds[inds] = 0
        
        for i in range(self.tourn_count):
            inds = TaskDist.rng.choice(self.population_size, size=self.tourn_size, p=free_inds / np.sum(free_inds))
            party_inds = inds[np.argsort(self.fitness()[inds])]

            for ind in party_inds:
                if TaskDist.rng.random() < self.tourn_p:
                    new_inds[i] = ind
                    break
            else:
                new_inds[i] = party_inds[-1]

            free_inds[new_inds[i]] = 0
        
        self.survivors = self.population[new_inds]
            
    def crossover(self):
        self.offsprings = []
        
        for _ in range(self.population.shape[0] - self.survivors.shape[0]):
            k1, k2 = TaskDist.rng.integers(0, self.individ_size // 2, size=1)[0], TaskDist.rng.integers(self.individ_size // 2, self.individ_size, size=1)[0]
            parent1, parent2 = TaskDist.rng.choice(self.survivors, size=2, replace=False)
            
            offspring1, offspring2 = parent1.copy(), parent2.copy()
            offspring1[k1:k2], offspring2[k1:k2] = offspring2[k1:k2], offspring1[k1:k2]

            self.offsprings.append(TaskDist.rng.choice([offspring1, offspring2], size=1)[0])
            
        self.offsprings = np.array(self.offsprings)
    
    def mutation(self):
        for i in range(self.offsprings.shape[0]):
            if TaskDist.rng.random() < self.mutation_prob:
                j = TaskDist.rng.integers(self.individ_size, size=1)
                self.offsprings[i, j] = TaskDist.rng.integers(self.dev_count, size=1)
    
    def step(self):
        self.selection()
        self.crossover()
        self.mutation()
        
        self.population = np.concatenate([self.survivors, self.offsprings], axis=0)
        self._fitness = None
        

In [401]:
taskDist = TaskDist(task_complexity, execution_time, dev_coefs, M, 500, N, 5, 10, 5, 0.7, 0.5)

In [402]:
pbar = tqdm.trange(10000)
pbar.set_description(f'{np.min(taskDist.fitness())}')

for _ in pbar:
    taskDist.step()
    pbar.set_description(f'{np.min(taskDist.fitness())}')

  0%|          | 0/10000 [00:00<?, ?it/s]

In [406]:
np.savetxt('output.txt', taskDist.population[np.argsort(taskDist.fitness())][0], newline = ' ', fmt='%.0f')

In [409]:
np.savetxt('output.txt', taskDist.population[np.argsort(taskDist.fitness())][0] + 1, newline = ' ', fmt='%.0f')

In [403]:
1e6 / 603.3 / 10000

0.1657550140891762